# Fake Page View generator

This notebook generates page views based on a three-hour dataset.

In [ ]:
%%bash
# Install the required Python 3 dependencies
python3 -m pip install kafka-python  # type: ignore

In [ ]:
# Extract archive of click data
import zipfile
import os

ARCHIVEF = "20180113-6-9-with-nulls.json.zip"
CLICKSF = "20180113-6-9-with-nulls.json"

if (not os.path.isfile(CLICKSF)):
    with zipfile.ZipFile(ARCHIVEF,"r") as zip_ref:
        zip_ref.extractall(".")

In [ ]:
# Send the ratings to the endpoint in order to test your implementation of the endpoint.
SENDMODE = "http"

# Send the ratings straight to Kafka in order to simulate a working endpoint.
# This allows you to test cleaning code without a working endpoint. 
# SENDMODE = "kafka"


In [ ]:
# Send click data
import json
import time
from datetime import datetime
import itertools

import requests
from kafka import KafkaProducer
from kafka.errors import KafkaError
from IPython.display import clear_output

producer = KafkaProducer(bootstrap_servers=['localhost:9092'])

orig_start = 0
replay_start = time.time() * 1000

counter = 0
i = 0

print("Simulating website traffic...")

with open(CLICKSF, 'r') as f:
    first = True
    start_ts = 0
    for line in itertools.islice(f, 1633):
        click = json.loads(line)

        orig_event = click["ts_ingest"] or 0
        
        if orig_event:
            curr_ts = time.time() * 1000

            if first:
                first = False
                orig_start = click["ts_ingest"]
            
        click['ts_ingest'] = round((orig_event - orig_start) + replay_start)

        if SENDMODE == "kafka":            
            future = producer.send('clicks', json.dumps(click).encode('utf-8'))
            record_metadata = future.get(timeout=10)
        else:
            requests.post("http://localhost:5000/clicks", data=json.dumps(click).encode("utf-8"), headers={'Content-Type':'application/json'})
        
        print("✓ {}".format(datetime.fromtimestamp(orig_event/1000.0)))
        if i > 1000:
            clear_output()
            i = 0
        i = i+1
